In [1]:
import pandas as pd
import regex as re

In [2]:
data = pd.read_csv(r'C:\Users\Seal\PycharmProjects\Another notebooks\log_week_end.csv', names=['rn', 'loguser', 'q'])

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
data.sample(5)

,rn,loguser,q
335860,321346,etl_2048,from tbl_53613
16979,538838,etl_528,"join tbl_15882,from tbl_317555,from tbl_317555"
105291,726600,etl_2048,from tbl_74885
489163,845826,etl_1151,from tbl_12145
366219,265388,etl_1151,from tbl_21615


Собираем в списки все команды

In [4]:
intos, froms, joins = [], [], []

In [5]:
for x in data.q.str.split(','):
    for g in x:
        if 'from' in g:
            froms.append(g)
        if 'join' in g:
            joins.append(g)
        if 'into' in g:
            intos.append(g)

Оставляем только название таблиц

In [6]:
intos_tbs = [re.search('tbl_\d*', x).group(0) for x in intos]
froms_tbs = [re.search('tbl_\d*', x).group(0) for x in froms]
joins_tbs = [re.search('tbl_\d*', x).group(0) for x in joins]

Делаем датафрейм с количество from (from только от пользователей dev) при помощи функции `create_df`

In [7]:
def create_df(input_data, column):
    df = pd.DataFrame(data = input_data, columns=['col']).groupby('col').agg({'col': 'count'})
    df.columns= [column]
    df.reset_index(inplace=True)
    return df

In [8]:
df1 = create_df(froms_tbs, 'count_froms')

**Делаем дф с кол-ом join**

In [9]:
df3 = create_df(joins_tbs, 'count_joins')

Объединим join и from при помощи `merge_df`:

In [10]:
def merge_df(data1, data2):
    data = data1.merge(data2, on ='col', how='outer')
    data = data.fillna(0)
    return data

In [11]:
df1 = merge_df(df1,df3)
df1['count_selects'] = df1['count_froms'] + df1['count_joins']
df1.head()

,col,count_froms,count_joins,count_selects
0,tbl_10000,24.0,6.0,30.0
1,tbl_10001,24.0,6.0,30.0
2,tbl_100026,6.0,0.0,6.0
3,tbl_100027,6.0,0.0,6.0
4,tbl_100028,6.0,0.0,6.0


**Делаем дф с кол-ом into**

In [12]:
df2 = create_df(intos_tbs, 'count_intos')
df = merge_df(df1,df2)
df['usefull'] = df['count_intos'] / df['count_selects']
df.head()

,col,count_froms,count_joins,count_selects,count_intos,usefull
0,tbl_10000,24.0,6.0,30.0,12.0,0.4
1,tbl_10001,24.0,6.0,30.0,12.0,0.4
2,tbl_100026,6.0,0.0,6.0,6.0,1.0
3,tbl_100027,6.0,0.0,6.0,6.0,1.0
4,tbl_100028,6.0,0.0,6.0,6.0,1.0


In [13]:
pd.set_option('mode.use_inf_as_na', True) #включаем опцию, где inf (бесконечность) считаем как NaN

In [16]:
df.sort_values(by = 'usefull', ascending = False, inplace = True)
df.sample(10, random_state = 100)

,col,count_froms,count_joins,count_selects,count_intos,usefull
9680,tbl_156562,24.0,0.0,24.0,6.0,0.25
25116,tbl_45774,1.0,0.0,1.0,1.0,1.00
18711,tbl_211502,2.0,0.0,2.0,1.0,0.50
15627,tbl_195522,24.0,6.0,30.0,0.0,0.00
21697,tbl_31280,1.0,0.0,1.0,1.0,1.00
25720,tbl_49291,12.0,0.0,12.0,6.0,0.50
11164,tbl_165468,6.0,0.0,6.0,0.0,0.00
18158,tbl_209286,5.0,0.0,5.0,0.0,0.00
7007,tbl_140256,10.0,0.0,10.0,0.0,0.00
16593,tbl_201648,18.0,6.0,24.0,6.0,0.25


# Леша брал в df не head(), a sample(), поэтому образцы у нас разные,конечно. Если это не ошибка и мы действительно будем брать именно образец, предлагаю зафиксировать random_state = 100, к примеру. 

In [19]:
df.query('col == "tbl_185730"') #это можно удалить, тут я проверяю, что данные одинаковые

,col,count_froms,count_joins,count_selects,count_intos,usefull
14223,tbl_185730,2.0,0.0,2.0,0.0,0.0


In [18]:
df.query('col == "tbl_92829"')

,col,count_froms,count_joins,count_selects,count_intos,usefull
33320,tbl_92829,15.0,10.0,25.0,0.0,0.0
